<a href="https://colab.research.google.com/github/BrunoBertolinoSC/teste/blob/main/Cambio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import requests
import json
import openpyxl
from openpyxl import load_workbook
import requests
from datetime import datetime, timedelta

In [52]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [61]:
planilha = load_workbook(filename = '/content/gdrive/MyDrive/Processo Seletivo/Faturamento_original.xlsx')

In [62]:
def get_historical_exchange_rate(date_str, currency):

    try:
        # Formatar a data e verificar se é um dia útil
        date_obj = datetime.strptime(date_str, '%Y-%m-%d').date()
        while date_obj.weekday() > 4:  # Ajustar para dias úteis (segunda a sexta)
            date_obj -= timedelta(days=1)

        formatted_date = date_obj.strftime('%Y%m%d')  # API exige o formato 'YYYYMMDD'

        # Fazer a requisição para a API
        url = f'https://economia.awesomeapi.com.br/json/daily/{currency}-BRL?start_date={formatted_date}&end_date={formatted_date}'
        response = requests.get(url)
        response.raise_for_status()  # Verifica se houve erro na requisição

        # Analisar a resposta
        data = response.json()
        if data:
            return float(data[0]['bid'])  # 'bid' é a cotação de compra
        return None

    except Exception as e:
        print(f"Erro ao buscar a cotação de {currency} para {date_str}: {e}")
        return None

In [63]:
def transforma_data(data_str):

    try:
        # Converte a string para um objeto datetime
        data_obj = datetime.strptime(data_str, "%d/%m/%Y")

        # Retorna a data formatada como yyyy-mm-dd
        return data_obj.strftime("%Y-%m-%d")
    except ValueError as e:
        print(f"Erro ao converter a data: {e}")
        return None


In [69]:

g = planilha['Página1']



last_column = g.max_column


next_column = last_column + 1

if g.cell(row=1, column=next_column).value is None:
  g.cell(row=1, column=next_column, value='Dolar')
  g.cell(row=1, column=next_column + 1, value='Euro')
  g.cell(row=1, column=next_column + 2, value='Faturamento em Dolar')
  g.cell(row=1, column=next_column + 3, value='Faturamento em Euro')


g.cell(row=2, column=next_column, value=get_historical_exchange_rate(transforma_data(str(g.cell(row=2, column= 1))), 'USD'))
g.cell(row=2, column=next_column + 1, value=get_historical_exchange_rate(transforma_data(str(g.cell(row=2, column= 1))), 'EUR'))
#g.cell(row=2, column=next_column + 2, value=((g.cell(row=2, column= 5)).value/get_historical_exchange_rate(transforma_data(str(g.cell(row=2, column= 1).value)), 'USD')))
#g.cell(row=2, column=next_column + 3, value=((g.cell(row=2, column= 5)).value/get_historical_exchange_rate(transforma_data(str(g.cell(row=2, column= 1).value)), 'EUR')))



Erro ao converter a data: time data "<Cell 'Página1'.A2>" does not match format '%d/%m/%Y'
Erro ao buscar a cotação de USD para None: strptime() argument 1 must be str, not None
Erro ao converter a data: time data "<Cell 'Página1'.A2>" does not match format '%d/%m/%Y'
Erro ao buscar a cotação de EUR para None: strptime() argument 1 must be str, not None


<Cell 'Página1'.V2>

In [70]:
planilha.save(filename = '/content/gdrive/MyDrive/Processo Seletivo/Faturamento_original.xlsx')